# Scraping KDD2019

In [1]:
import requests
from bs4 import BeautifulSoup

In [3]:
papers = {}

target_url = 'http://www.kdd.org/kdd2019/accepted-papers'
r = requests.get(target_url)
soup = BeautifulSoup(r.text, 'lxml')
for i, item in enumerate( soup.select('a[href^="https://www.kdd.org/kdd2019/accepted-papers/view/"]') ):
    if i % 20 == 0: print( i )
    title = item.get_text()
    url = item.get('href')
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    s = soup.find('section', {'class': 'g-pt-50 g-pb-100'})

    author = s.p.text
    author_list = [ n.strip() for n in author.split(';')[:-1]]
    abst = s.find_all('p')[1].text
    try:
        yid = s.find('iframe').attrs['src'].split('/')[-1]
    except:
        yid = 'None'
    try:
        pdf = s.find('a', {'class': 'btn u-btn-primary g-brd-2 g-brd-white g-font-size-13 g-rounded-50 g-pl-20 g-mb-20 g-pr-15 g-py-9'}).get('href')
    except:
        pdf = 'None'
    
    paper = {}
    paper['title'] = title
    paper['author'] = author_list
    paper['abst'] = abst
    paper['yid'] = yid
    paper['pdf'] = pdf
    papers[i] = paper

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320


In [11]:
len(papers)

321

# Get scrapbox import file

In [13]:

import json
from pprint import pprint

f = open( '../scrapbox_file/kdd2019-paperlist.json', 'r' )
j = json.load( f )

for i, paper in papers.items():

    if i > 319 : continue

    page = {}
    page['title'] = paper['title']

    page['lines'] = []
    page['lines'].append( paper['title'] )

    page['lines'].append( '' )   
    aut = ''
    for a in paper['author']:
        a = a.replace(' ', '_')
        aut += '#%s; ' % a
    page['lines'].append( aut )
    
    page['lines'].append( '' )
    page['lines'].append( paper['abst'] )

    if paper['yid'] == 'None':
        page['lines'].append( '' )
        page['lines'].append( 'The video does not exist' )    
    else:
        page['lines'].append( '' )
        page['lines'].append( '[https://www.youtube.com/watch?v=%s]' % paper['yid'] )    
    
    if paper['pdf'] == 'None':
        page['lines'].append( '' )
        page['lines'].append( 'The PDF does not exist' )  
    else:
        page['lines'].append( '' )
        page['lines'].append( 'PDF url: %s' % paper['pdf'] )

    page['lines'].append( '' )
    page['lines'].append( '一言でいうと(３行以内)' )
    page['lines'].append( '' )
    page['lines'].append( '新規性・差分' )
    page['lines'].append( '' )
    page['lines'].append( '手法' )
    page['lines'].append( '' )
    page['lines'].append( '結果' )
    page['lines'].append( '' )
    page['lines'].append( 'コメント' )

    page['lines'].append( '' )
    page['lines'].append( '[/icons/hr.icon]' )
    page['lines'].append( 'External URLs' )
    
    page['lines'].append( '' )
    page['lines'].append( '[/icons/hr.icon]' )
    page['lines'].append( 'Tags' )
    page['lines'].append( '#kdd2019' )
        
    j['pages'].append( page )
    
fw = open( '../scrapbox_file/kdd2019-paperlist-mod.json', 'w' )
json.dump( j, fw, indent=2 )